In [8]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch 
from torch.nn.utils.rnn import pad_sequence
from transformers.models.mbart.modeling_mbart import shift_tokens_right
# Load pre-trained MBART model and tokenizer (MBART-50 for multilingual tasks)
model_name = "../mbart_model"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Example batch of input sentences in various languages
batch_sentences = [
    "Hello, how are you?",   # English
    "What have you been up to recently?", # French
    "Do you want to go for a run?",    # Spanish
]



# Tokenize the input batch of sentences
inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True)
inputs = shift_tokens_right(inputs.input_ids, tokenizer.pad_token_id)
print(inputs )
# Generate translations (for example, to French) or any other target language
# Specify the target language for the model to generate in
forced_bos_token_id = tokenizer.lang_code_to_id["zh_CN"]

# Perform inference with the model to generate translations
outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], 
                         forced_bos_token_id=forced_bos_token_id)

predictions = [] 
for i in range(len(outputs)): 
    predictions.append(outputs[i, :])

pad_tensor = torch.ones(200-len(predictions[0]))
predictions[0] = torch.cat((predictions[0],pad_tensor.long()),dim = 0)
predictions = pad_sequence(predictions,batch_first=True,padding_value=1)

# Decode the generated outputs back to text
translated_sentences = tokenizer.batch_decode(predictions, skip_special_tokens=True)

# Print the generated translations
for i, translation in enumerate(translated_sentences):
    print(f"Original: {batch_sentences[i]}")
    print(f"Translated: {translation}")
    print()

loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file ../mbart_model/config.json
Model config MBartConfig {
  "_name_or_path": "facebook/mbart-large-50-many-to-many-mmt",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_c

tensor([[     2, 250004,  35378,      4,   3642,    621,    398,     32,      2,
              1,      1],
        [     2, 250004,   4865,    765,    398,   2809,   1257,     47,  78684,
             32,      2],
        [     2, 250004,    984,    398,   3444,     47,    738,    100,     10,
          11675,     32]])


IndexError: too many indices for tensor of dimension 2

## Testing LLM adaptor 2 shape


In [27]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

# Define a dummy PAD_IDX for padding purposes
PAD_IDX = 0

class LLMAdapter2(nn.Module):
    '''
    LLM adapter aims to capture temporal relations and transform 32 tokens into 1024 tokens.
    This version introduces an additional projection layer between the two convolution layers.
    '''
    def __init__(self, num_tokens=32, hidden_dim=1024, kernel_size=5):
        super(LLMAdapter2, self).__init__()
        
        # Store parameters
        self.num_tokens = num_tokens
        self.hidden_dim = hidden_dim
        
        # First projection from input tokens to hidden_dim/2
        self.proj = nn.Linear(self.num_tokens, self.hidden_dim // 2)

        # First convolutional block
        self.conv_block_1 = nn.Sequential(
            nn.Conv1d(self.hidden_dim // 2, self.hidden_dim // 2, kernel_size=kernel_size, stride=1, padding=0),
            nn.BatchNorm1d(self.hidden_dim // 2),
            nn.ReLU(inplace=True),
            nn.AvgPool1d(kernel_size=2, ceil_mode=False)
        )

        # New projection layer between convolution layers
        self.intermediate_proj = nn.Linear(self.hidden_dim // 2, self.hidden_dim)

        # Second convolutional block
        self.conv_block_2 = nn.Sequential(
            nn.Conv1d(self.hidden_dim, self.hidden_dim, kernel_size=kernel_size, stride=1, padding=0),
            nn.BatchNorm1d(self.hidden_dim),
            nn.ReLU(inplace=True),
            nn.AvgPool1d(kernel_size=2, ceil_mode=False)
        )


    def forward(self, x, src_length):
        # Input shape: (batch_size, num_frames, num_tokens)
        
        # Split the input into individual batches according to src_length
        start = 0
        x_batch = []
        for length in src_length:
            end = start + length
            x_batch.append(x[start:end])
            start = end
        
        # Pad sequences to ensure uniform batch sizes
        x = pad_sequence(x_batch, padding_value=PAD_IDX, batch_first=True)
        print(f"After padding: {x.shape}")  # Check padding result (batch_size, num_frames, num_tokens)
        
        # Apply the initial projection layer
        x = self.proj(x)  # Shape: (batch_size, num_frames, hidden_dim / 2)
        print(f"After initial projection: {x.shape}")  # Should be (batch_size, num_frames, 512)
        
        # Permute to (batch_size, hidden_dim / 2, num_frames) for Conv1d
        x = x.permute(0, 2, 1)
        print(f"After permute (before first conv): {x.shape}")  # Should be (batch_size, 512, num_frames)
        
        # First convolutional block
        x = self.conv_block_1(x)  # Shape: (batch_size, hidden_dim / 2, reduced_num_frames)
        print(f"After first conv block: {x.shape}")  # Check after first conv
        
        # Apply the intermediate projection layer
        x = x.permute(0, 2, 1)  # Back to (batch_size, reduced_num_frames, hidden_dim / 2)
        x = self.intermediate_proj(x)  # Shape: (batch_size, reduced_num_frames, hidden_dim)
        print(f"After intermediate projection: {x.shape}")  # Should be (batch_size, reduced_num_frames, 1024)
        x = x.permute(0, 2, 1)  # Back to (batch_size, hidden_dim, reduced_num_frames)
        print(f"After permute (before second conv): {x.shape}")  # Check before second conv
        
        # Second convolutional block
        x = self.conv_block_2(x)  # Shape: (batch_size, hidden_dim, further_reduced_num_frames)
        print(f"After second conv block: {x.shape}")  # Check after second conv
        
        # Convert back to (batch_size, further_reduced_num_frames, hidden_dim)
        x = x.permute(0, 2, 1)
        print(f"Final output shape: {x.shape}")  # Should be (batch_size, further_reduced_num_frames, 1024)

        return x

# Create an instance of LLMAdapter2
model = LLMAdapter2()

# Test input
batch_size = 10
num_frames = 50  # Let's assume each sequence has 15 frames
num_tokens = 32  # As specified in the model

# Random test tensor simulating a batch of 10 sequences, each with 15 frames and 32 tokens
test_input = torch.rand((batch_size * num_frames, num_tokens))

# Source lengths for each batch (assuming all sequences have 15 frames)
src_length = torch.tensor([num_frames] * batch_size)

# Forward pass
output = model(test_input, src_length)

After padding: torch.Size([10, 50, 32])
After initial projection: torch.Size([10, 50, 512])
After permute (before first conv): torch.Size([10, 512, 50])
After first conv block: torch.Size([10, 512, 23])
After intermediate projection: torch.Size([10, 23, 1024])
After permute (before second conv): torch.Size([10, 1024, 23])
After second conv block: torch.Size([10, 1024, 9])
Final output shape: torch.Size([10, 9, 1024])


## Testing LLM adaptor 3 shape

In [2]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

# Define a dummy PAD_IDX for this example
PAD_IDX = 0

class LLMAdapter3(nn.Module):
    def __init__(self, num_tokens=32, hidden_dim=1024, kernel_size=5):
        super(LLMAdapter3, self).__init__()
        self.num_tokens = num_tokens
        self.hidden_dim = hidden_dim
        
        # Temporal convolution over the time dimension
        self.temporal_conv = nn.Sequential(
            nn.Conv1d(self.num_tokens, self.num_tokens * 2, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm1d(self.num_tokens * 2),  # Channels must match Conv1d output channels
            nn.ReLU(inplace=True),
            # Reduce kernel size for pooling to avoid sequence collapse
            nn.AvgPool1d(kernel_size=1, ceil_mode=False),  

            nn.Conv1d(self.num_tokens * 2, self.num_tokens * 4, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm1d(self.num_tokens * 4),  # Channels must match Conv1d output channels
            nn.ReLU(inplace=True),
            nn.AvgPool1d(kernel_size=1, ceil_mode=False)  # Adjusted pooling to avoid reducing size to zero
        )
        
        # Final projection layer
        self.final_proj = nn.Sequential(
            nn.Linear(self.num_tokens * 4, self.hidden_dim)
        )
        self.out = nn.Sequential(nn.BatchNorm1d(self.hidden_dim),
            nn.ReLU(inplace=True))

    def forward(self, x, src_length):
        start = 0
        x_batch = []
        for length in src_length:
            end = start + length
            x_batch.append(x[start:end])
            start = end
        print(f"Before padding: {x.shape}") 
        x = pad_sequence(x_batch, padding_value=PAD_IDX, batch_first=True)
        print(f"After padding: {x.shape}")  # Print shape after padding
        
        # Permute to match Conv1d expected shape: (batch_size, channels, sequence_length)
        x = x.permute(0, 2, 1)
        print(f"After permute: {x.shape}")  # Shape should now be (batch_size, num_tokens, num_frames)
        
        # Apply temporal convolution
        x = self.temporal_conv(x)
        print(f"After temporal_conv: {x.shape}")  # Check shape after convolution
        
        # Permute back to (batch_size, sequence_length, hidden_dim)
        x = x.permute(0, 2, 1)
        print(f"After second permute: {x.shape}")  # Shape should be (batch_size, num_frames, num_tokens*4)
        
        # Apply final projection (we need to flatten or reshape input to match Linear input requirements)
        batch_size, seq_len, hidden_dim = x.shape
        x = self.final_proj(x)
        #x = self.final_proj(x.reshape(batch_size * seq_len, hidden_dim))
        print(f"After final_proj: {x.shape}")  # Check final shape

        print(f"before out shape : {x.shape}")
        x = self.out(x.permute(0, 2, 1)).permute(0, 2, 1)
        return x

# Create an instance of LLMAdapter3
model = LLMAdapter3()

# Test input
batch_size = 10
num_frames = 50 # Let's assume each sequence has 15 frames
num_tokens = 32  # As specified in the model

# Random test tensor simulating a batch of 10 sequences, each with 15 frames and 32 tokens
test_input = torch.rand((batch_size * num_frames, num_tokens))

# Source lengths for each batch (assuming all sequences have 15 frames)
src_length = torch.tensor([num_frames] * batch_size)

# Forward pass
output = model(test_input, src_length)

Before padding: torch.Size([500, 32])
After padding: torch.Size([10, 50, 32])
After permute: torch.Size([10, 32, 50])
After temporal_conv: torch.Size([10, 128, 42])
After second permute: torch.Size([10, 42, 128])
After final_proj: torch.Size([10, 42, 1024])
before out shape : torch.Size([10, 42, 1024])


### Create tokens dataset 

### Test some generation

In [2]:
import torch 
import torch.nn as nn
from omegaconf import OmegaConf
import torch.distributed
from train_sign_utils import * 
import os 
from accelerate import Accelerator
from logger import setup_logger
from accelerate.utils import set_seed
import sys 
from transformers import MBart50Tokenizer
import torch.multiprocessing as mp

In [9]:
## Take in a configuration 
import train_sign_utils
import signdata
import seq_model
from imp import reload
reload(seq_model)
reload(signdata)
reload(train_sign_utils)
from train_sign_utils import create_model, create_signloader




config = OmegaConf.load("./configs/Sign2Text_CSL_config_v3.yaml")


output_dir = config.experiment.output_dir

os.makedirs(output_dir, exist_ok=True)
config.experiment.logging_dir = os.path.join(output_dir, "logs")
# Load the model 
accelerator = Accelerator(
        gradient_accumulation_steps=config.training.gradient_accumulation_steps,
        mixed_precision=config.training.mixed_precision,
        project_dir=config.experiment.logging_dir,
        split_batches=False
    )


logger = setup_logger(name="Sign2Text", log_level="INFO",
    output_file=f"{output_dir}/log{accelerator.process_index}.txt")

# We need to initialize the trackers we use, and also store our configuration.

# If passed along, set the training seed now.
if config.training.seed is not None:
    set_seed(config.training.seed, device_specific=True)

# Create model 
model, ema_model = create_model(config, logger, accelerator)
# Create signloaders 
tokenizer = MBart50Tokenizer.from_pretrained(config.training.tokenizer,
                                            src_lang=config.dataset.lang,
                                              tgt_lang= config.dataset.lang)
train_dataloader, dev_dataloader, test_dataloader = create_signloader(config, logger, accelerator, tokenizer, "cpu")





[10/24 22:58:22 Sign2Text]: Creating model and loss module.


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=False)
  warnings.warn(


Titok weights loaded successfully from: TiTok_weights/ema_model/pytorch_model.bin
TiTok weights are frozen chowwww!
MBart encoder weights are graded!
MBart decoder weights are frozen!
Number of trainable parameters: 152209408
[10/24 22:58:37 Sign2Text]: loading weight from ./frozen_sign2text/ema_model/pytorch_model.bin, msg: <All keys matched successfully>


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[10/24 22:58:39 Sign2Text]: Creating Signloaders. Batch_size = 1
train dataloader done!
dev dataloader done!
train dataloader done!


In [11]:
train_dataloader.dataset[0][1].shape

for i, (src, tgt) in enumerate(tqdm(train_dataloader, desc=f"Generating!")):

    batch = src['input_ids']
    src_length = src['src_length_batch']
    tgt_attn = tgt.attention_mask
    tgt_input = tgt['input_ids']
    input_attn = src['attention_mask']
    print(batch.shape)
    print(src_length)
    print(tgt_attn.shape)
    print(tgt_input.shape)
    print(input_attn.shape)
    break


Generating!:   0%|          | 0/18401 [00:00<?, ?it/s]

torch.Size([120, 32])
tensor([120], device='cuda:0')
torch.Size([1, 11])
torch.Size([1, 11])
torch.Size([1, 27])


Generating!:   0%|          | 0/18401 [00:13<?, ?it/s]


In [16]:
def translate_images(model, images, tgt_labels, input_attn, src_length, config, accelerator,  logger, tokenizer): 

    logger.info("Translating images...")
    model = accelerator.unwrap_model(model).to("cuda")
    images = torch.clone(images)
    
    # Set appropriate dtype based on mixed precision
    dtype = torch.float32
    if accelerator.mixed_precision == "fp16":
        dtype = torch.float16
    elif accelerator.mixed_precision == "bf16":
        dtype = torch.bfloat16

    with torch.no_grad(): 

        # Directly generate translations using model.generate
        output = model.generate(
            src_input=images, 
            src_attn=input_attn, 
            src_length=src_length,
            max_new_tokens=150, 
            num_beams=4, 
            decoder_start_token_id=tokenizer.lang_code_to_id[config.dataset.lang]
        )

        
        # Use tokenizer to decode generated token IDs to translations
        pred_translations = tokenizer.batch_decode(output, skip_special_tokens=True)

        # Decode the target labels (ground truth)
        gt_translations = tokenizer.batch_decode(tgt_labels, skip_special_tokens=True)

    
    return pred_translations, gt_translations

In [17]:
# Load the dataset and dataloader 
for i, (src, tgt) in enumerate(tqdm(train_dataloader, desc=f"Generating!")):
    model.to("cuda")
    model.eval()
    batch = src['input_ids']
    src_length = src['src_length_batch']
    tgt_attn = tgt.attention_mask
    tgt_input = tgt['input_ids']
    input_attn = src['attention_mask']

    images = batch.to(
                accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
            )
    #print(f"imagges type: {images.type()}")
    tgt_input = tgt['input_ids'].to(
            accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
        )
    input_attn = input_attn.to(
            accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
        )
    tgt_attn = tgt_attn.to(
            accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
        )
    
    original_images= torch.clone(images)
    print("images shape ", original_images.shape)
    print("input attn shape", input_attn.shape)
    print("tgt attn shape", tgt_attn.shape)
    print("src length shape", src_length.shape)
    print("tgt input shape", tgt_input.shape)
    # Save a batch of translated images to check by reading

    fw_out = model( src_input = original_images,tgt_input = tgt_input, src_attn=input_attn, tgt_attn = tgt_attn, src_length = src_length)
    print(f"fw_out ")
    pred, gt = translate_images(
        model=model,
        images=images,
        tgt_labels=tgt_input,
        input_attn=input_attn, 
        src_length=src_length,
        config=config,
        accelerator=accelerator,
        logger=logger, 
        tokenizer=tokenizer
    )
    break 


print(f"predictions: {pred}")
print(f"ground truth: {gt}")

Generating!:   0%|          | 0/18401 [00:00<?, ?it/s]

images shape  torch.Size([96, 3, 256, 256])
input attn shape torch.Size([1, 21])
tgt attn shape torch.Size([1, 11])
src length shape torch.Size([1])
tgt input shape torch.Size([1, 11])
fw_out 
[10/24 22:24:31 Sign2Text]: Translating images...


Generating!:   0%|          | 0/18401 [00:32<?, ?it/s]


IndexError: too many indices for tensor of dimension 2

In [50]:
model

SignModel(
  (titok): TiTok(
    (encoder): TiTokEncoder(
      (patch_embed): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (transformer): ModuleList(
        (0-23): 24 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
        )
      )
      (ln_post): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (conv_out): Conv2d(1024, 12, kernel_size=(1, 1), stride=(1, 1))
    )
    (decoder)

### Checking LLM adaptation and linking it to gloss

In [14]:
 ## Reading in the labelled annotations
import pickle
def read_CSL_annotations(CSL_annot_path):
    with open(CSL_annot_path, 'rb') as f:
        data = pickle.load(f)
    return data
train_labels = read_CSL_annotations("../../CSL-Daily/sentence_label/processed/labels_train.pkl") 
dev_labels = read_CSL_annotations("../../CSL-Daily/sentence_label/processed/labels_dev.pkl")
test_labels = read_CSL_annotations("../../CSL-Daily/sentence_label/processed/labels_test.pkl")
combined_labels = read_CSL_annotations("../../CSL-Daily/sentence_label/csl2020ct_v2.pkl")

def find_entry_by_name(data, name):
    # Iterate through the list of dictionaries in 'info'
    for entry in data['info']:
        if entry['name'] == name:
            return entry
    return None


def get_gloss(combined_annotations, name): 
  # find entry in combined_annotations 
  entry = find_entry_by_name(combined_labels, name)
  gloss_entry = entry['label_gloss']
  return gloss_entry


def match_frames_w_gloss(tensor_lst, gloss): 
  # divide the frames into glosses
  num_tensors = len(tensor_lst)
  num_gloss = len(gloss)
  tensors_per_gloss = num_tensors//num_gloss

  # create a dictionary to hold the frames referring to a gloss
  gloss_dict = {}
  for i, g in enumerate(gloss):
    if i == num_gloss-1: 
      gloss_dict[g] = [tensor_lst[i*tensors_per_gloss:]]
    else: 
      gloss_dict[g] = [tensor_lst[i*tensors_per_gloss:(i+1)*tensors_per_gloss]]
  
  return gloss_dict

def find_and_combine_glossdicts(phase, num_samples = None , name_lst= None , dir="../../CSL-Daily/sentence/frames_512x512"):
  gloss_dict_lst = {}
  ## make assertions to prevent error 
  assert phase in ['train', 'dev', 'test'], "Phase must be either train, dev or test"
  assert num_samples is not None or name_lst is not None, "Either number of samples or name list must be provided"

  if name_lst is None:
    video_lst = os.listdir(f"{dir}/{phase}")
    # Sample random number of videos
    name_lst =  random.sample(video_lst, num_samples)

  ## Gather various gloss dicts 
  for name in name_lst: 
    gloss = get_gloss(combined_labels, name)
    tensor_lst = gather_vid_emb(name, phase)
    gloss_dict_lst[name] = match_frames_w_gloss(tensor_lst, gloss)

  # Comebine gloss dicts 
  combined_gloss_dict = {}
  for name in name_lst: 
    for k, v in gloss_dict_lst[name].items(): 
      if k in combined_gloss_dict: 
        combined_gloss_dict[k].extend(v)
      else: 
        combined_gloss_dict[k] = v
  return gloss_dict_lst, combined_gloss_dict



In [14]:
'''Checking after LLM adaptation'''
 
# Separate TikTok model and LLM adapter for testing
titok = model.titok
adapter = model.adapter
titok.eval()
adapter.eval()
# run the video frames through the TikTok model 
# Load the dataset and dataloader 
with torch.no_grad():
    for i, (src, tgt) in enumerate(tqdm(train_dataloader, desc=f"adapting train!")):
        phase = "train"
        titok.to("cuda")
        adapter.to("cuda")
        batch = src['input_ids']
        src_length = src['src_length_batch']
        tgt_attn = tgt.attention_mask
        tgt_input = tgt['input_ids']
        input_attn = src['attention_mask']

        images = batch.to(
                    accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
                )
        #print(f"imagges type: {images.type()}")
        tgt_input = tgt['input_ids'].to(
                accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
            )
        input_attn = input_attn.to(
                accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
            )
        tgt_attn = tgt_attn.to(
                accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
            )
        
        original_images= torch.clone(images)
        # print(f"names: {src['name_batch']}")
        # print("images shape ", original_images.shape)
        # print("input attn shape", input_attn.shape)
        # print("tgt attn shape", tgt_attn.shape)
        # print("src length shape", src_length.shape)
        # print("tgt input shape", tgt_input.shape)
        # Save a batch of translated images to check by reading
        encoded_tokens = titok.encode(x=original_images)[1]['min_encoding_indices'].squeeze()
        #print(f"encoded tokens shape: {encoded_tokens.shape}")
        hidden_values = adapter(encoded_tokens.float(), src_length)
        #print(f"hidden values shape: {hidden_values.shape}")

        for i,  hid_val in enumerate(hidden_values): 
            hid_val = hid_val[input_attn[i]==1]
            #print(f"hid val: {hid_val.shape}")

            for j, token in enumerate(hid_val): 
                # save the token as a .pth file 
                path = os.path.join(config.dataset.img_path,phase )
                final_path = os.path.join(path, src['name_batch'][i])
                #print(final_path)
                torch.save(token, f"{final_path}/aft_adapter_{j}.pth")

    

    # Save newly compressed and temporal conv as .pth files and also consider the gloss used for the translation
    for i, (src, tgt) in enumerate(tqdm(dev_dataloader, desc=f"adapting dev!")):
        phase = "dev"
        titok.to("cuda")
        adapter.to("cuda")
        batch = src['input_ids']
        src_length = src['src_length_batch']
        tgt_attn = tgt.attention_mask
        tgt_input = tgt['input_ids']
        input_attn = src['attention_mask']

        images = batch.to(
                    accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
                )
        #print(f"imagges type: {images.type()}")
        tgt_input = tgt['input_ids'].to(
                accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
            )
        input_attn = input_attn.to(
                accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
            )
        tgt_attn = tgt_attn.to(
                accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
            )
        
        original_images= torch.clone(images)
        # print(f"names: {src['name_batch']}")
        # print("images shape ", original_images.shape)
        # print("input attn shape", input_attn.shape)
        # print("tgt attn shape", tgt_attn.shape)
        # print("src length shape", src_length.shape)
        # print("tgt input shape", tgt_input.shape)
        # Save a batch of translated images to check by reading
        encoded_tokens = titok.encode(x=original_images)[1]['min_encoding_indices'].squeeze()
        #print(f"encoded tokens shape: {encoded_tokens.shape}")
        hidden_values = adapter(encoded_tokens.float(), src_length)
        #print(f"hidden values shape: {hidden_values.shape}")

        for i,  hid_val in enumerate(hidden_values): 
            hid_val = hid_val[input_attn[i]==1]
            

            for j, token in enumerate(hid_val): 
                # save the token as a .pth file 
                path = os.path.join(config.dataset.img_path,phase )
                final_path = os.path.join(path, src['name_batch'][i])
                #print(final_path)
                torch.save(token, f"{final_path}/aft_adapter_{j}.pth")

    

    # Save newly compressed and temporal conv as .pth files and also consider the gloss used for the translation
    for i, (src, tgt) in enumerate(tqdm(test_dataloader, desc=f"adapting test!")):
        phase = "test"
        titok ,adapter= accelerator.prepare(titok,adapter)
        batch = src['input_ids']
        src_length = src['src_length_batch']
        tgt_attn = tgt.attention_mask
        tgt_input = tgt['input_ids']
        input_attn = src['attention_mask']

        images = batch.to(
                    accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
                )
        #print(f"imagges type: {images.type()}")
        tgt_input = tgt['input_ids'].to(
                accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
            )
        input_attn = input_attn.to(
                accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
            )
        tgt_attn = tgt_attn.to(
                accelerator.device, memory_format=torch.contiguous_format, non_blocking=True
            )
        
        original_images= torch.clone(images)
        # print(f"names: {src['name_batch']}")
        # print("images shape ", original_images.shape)
        # print("input attn shape", input_attn.shape)
        # print("tgt attn shape", tgt_attn.shape)
        # print("src length shape", src_length.shape)
        # print("tgt input shape", tgt_input.shape)
        # Save a batch of translated images to check by reading
        encoded_tokens = titok.encode(x=original_images)[1]['min_encoding_indices'].squeeze()
        #print(f"encoded tokens shape: {encoded_tokens.shape}")
        hidden_values = adapter(encoded_tokens.float(), src_length)
        #print(f"hidden values shape: {hidden_values.shape}")

        for i,  hid_val in enumerate(hidden_values): 
            hid_val = hid_val[input_attn[i]==1]
            #print(f"hid val: {hid_val.shape}")
            

            for j, token in enumerate(hid_val): 
                # save the token as a .pth file 
                path = os.path.join(config.dataset.img_path,phase )
                final_path = os.path.join(path, src['name_batch'][i])
                #print(final_path)
                torch.save(token, f"{final_path}/aft_adapter_{j}.pth")

    

    # Save newly compressed and temporal conv as .pth files and also consider the gloss used for the translation


adapting train!:   0%|          | 3/18401 [08:04<887:43:20, 173.70s/it]

### Testing pretrained GFSLT-VLP models 

In [3]:
'''Visual Encoder part'''
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
visual_encoder_path = "../pretrain_models/mytran"

trans_encoder = MBartForConditionalGeneration.from_pretrained(visual_encoder_path)
print(trans_encoder)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(2454, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(2454, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-2): 3 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (fin

As shown in the printed model architecture, the model has both an encoder and decoder layers. However, it only has 3 of each. In the GFSLT-VLP, only the encoder is taken

In [5]:
'''Text transformer'''

text_transformer_path = "../pretrain_models/MBart_proun"
text_transformer = MBartForConditionalGeneration.from_pretrained(text_transformer_path)
print(text_transformer)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(2454, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(2454, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
  

In [4]:
import torch
from transformers.models.mbart.modeling_mbart import shift_tokens_right

# Define our example inputs
target_ids = torch.tensor([[101, 200, 300, 400, 500, 102]])  # Shape (1, 6)

# Shift tokens with decoder start token <bos> (100)
shifted_target_ids = shift_tokens_right(target_ids, pad_token_id=1)

print("Original target_ids:", target_ids)
print("Shifted decoder_input_ids:", shifted_target_ids)

Original target_ids: tensor([[101, 200, 300, 400, 500, 102]])
Shifted decoder_input_ids: tensor([[102, 101, 200, 300, 400, 500]])


In [2]:
from transformers import MBart50Tokenizer
import torch
from transformers.models.mbart.modeling_mbart import shift_tokens_right
# Initialize the MBart50 tokenizer
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = "zh_CN"  # Set source language to Simplified Chinese

# Example Chinese sentence
sentence = "今天是个好天气。"  # "Today is a good weather."

# Tokenize the sentence
tokenized = tokenizer(sentence, return_tensors="pt", padding=True)
input_ids = tokenized['input_ids']

# Display the token IDs and decoded sentence to verify
print("Token IDs:", input_ids)
print("Decoded Sentence:", tokenizer.decode(input_ids[0], skip_special_tokens=True))

# Shift tokens with decoder start token <bos> (100)
shifted_input_ids = shift_tokens_right(input_ids, pad_token_id=1)

print("\nShifted Token IDs:", shifted_input_ids)
print("Shifted Decoded Sentence:", tokenizer.decode(shifted_input_ids[0], skip_special_tokens=True))

/opt/homebrew/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Token IDs: tensor([[250025,  61168,  41779,   1322,  70871,     30,      2]])
Decoded Sentence: 今天是个好天气。

Shifted Token IDs: tensor([[     2, 250025,  61168,  41779,   1322,  70871,     30]])
Shifted Decoded Sentence: 今天是个好天气。
